In [3]:
library(dplyr) 
library(RSQLite) 
my_db <- src_sqlite("flights_db.sqlite3", create = T) 

In [4]:
library(nycflights13)
flights_sqlite <- copy_to(my_db, flights, temporary = FALSE, indexes = list(
c("year", "month", "day"), "carrier", "tailnum"))

In [5]:
flights_sqlite <- tbl(nycflights13_sqlite(), "flights")

Caching nycflights db at /tmp/RtmpJZUcZp/nycflights13.sqlite
Creating table: airlines
Creating table: airports
Creating table: flights
Creating table: planes
Creating table: weather


In [6]:
flights_sqlite

Source:   query [?? x 19]
Database: sqlite 3.11.1 [/tmp/RtmpJZUcZp/nycflights13.sqlite]

    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1   2013     1     1      517            515         2      830            819
2   2013     1     1      533            529         4      850            830
3   2013     1     1      542            540         2      923            850
4   2013     1     1      544            545        -1     1004           1022
5   2013     1     1      554            600        -6      812            837
6   2013     1     1      554            558        -4      740            728
7   2013     1     1      555            600        -5      913            854
8   2013     1     1      557            600        -3      709            723
9   2013     1     1      557            600        -3      838            846
10  2013     1     1      558            6

In [7]:
select(flights_sqlite,year:day,dep_time:arr_time)

Source:   query [?? x 7]
Database: sqlite 3.11.1 [/tmp/RtmpJZUcZp/nycflights13.sqlite]

    year month   day dep_time sched_dep_time dep_delay arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>
1   2013     1     1      517            515         2      830
2   2013     1     1      533            529         4      850
3   2013     1     1      542            540         2      923
4   2013     1     1      544            545        -1     1004
5   2013     1     1      554            600        -6      812
6   2013     1     1      554            558        -4      740
7   2013     1     1      555            600        -5      913
8   2013     1     1      557            600        -3      709
9   2013     1     1      557            600        -3      838
10  2013     1     1      558            600        -2      753
# ... with more rows

In [8]:
filter(flights_sqlite,sched_dep_time>1000 & dep_time>1100)

Source:   query [?? x 19]
Database: sqlite 3.11.1 [/tmp/RtmpJZUcZp/nycflights13.sqlite]

    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1   2013     1     1     1101           1043        18     1345           1332
2   2013     1     1     1103           1100         3     1410           1421
3   2013     1     1     1105           1047        18     1428           1405
4   2013     1     1     1107           1115        -8     1305           1310
5   2013     1     1     1109           1115        -6     1402           1425
6   2013     1     1     1111           1115        -4     1222           1226
7   2013     1     1     1112           1100        12     1440           1438
8   2013     1     1     1113           1115        -2     1318           1315
9   2013     1     1     1120           1125        -5     1330           1325
10  2013     1     1     1123           11

In [9]:
arrange(flights_sqlite, sched_dep_time )  #Arranges according to ascending order

Source:   query [?? x 19]
Database: sqlite 3.11.1 [/tmp/RtmpJZUcZp/nycflights13.sqlite]

    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1   2013     7    27       NA            106        NA       NA            245
2   2013     1     2      458            500        -2      703            650
3   2013     1     3      458            500        -2      650            650
4   2013     1     4      456            500        -4      631            650
5   2013     1     5      458            500        -2      640            650
6   2013     1     6      458            500        -2      718            650
7   2013     1     7      454            500        -6      637            648
8   2013     1     8      454            500        -6      625            648
9   2013     1     9      457            500        -3      647            648
10  2013     1    10      450            5

In [10]:
mutate(flights_sqlite, speed = air_time / distance)

Source:   query [?? x 20]
Database: sqlite 3.11.1 [/tmp/RtmpJZUcZp/nycflights13.sqlite]

    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1   2013     1     1      517            515         2      830            819
2   2013     1     1      533            529         4      850            830
3   2013     1     1      542            540         2      923            850
4   2013     1     1      544            545        -1     1004           1022
5   2013     1     1      554            600        -6      812            837
6   2013     1     1      554            558        -4      740            728
7   2013     1     1      555            600        -5      913            854
8   2013     1     1      557            600        -3      709            723
9   2013     1     1      557            600        -3      838            846
10  2013     1     1      558            6

In [11]:
summarise(flights_sqlite, delay = mean(dep_time))

Source:   query [?? x 1]
Database: sqlite 3.11.1 [/tmp/RtmpJZUcZp/nycflights13.sqlite]

    delay
    <dbl>
1 1349.11

In [12]:
c1 <- filter(flights_sqlite, year == 2013, month == 5, day == 15)
c2 <- select(c1, year, month, day, carrier, dep_delay, air_time, distance)
c3 <- mutate(c2, speed = distance / air_time * 60)
c4 <- arrange(c3, year, month, day, carrier)
c4

Source:   query [?? x 8]
Database: sqlite 3.11.1 [/tmp/RtmpJZUcZp/nycflights13.sqlite]

    year month   day carrier dep_delay air_time distance    speed
   <int> <int> <int>   <chr>     <dbl>    <dbl>    <dbl>    <dbl>
1   2013     5    15      9E       -10       91      569 375.1648
2   2013     5    15      9E        -9       69      340 295.6522
3   2013     5    15      9E        -1       87      509 351.0345
4   2013     5    15      9E        -8      161     1029 383.4783
5   2013     5    15      9E        -3      122      740 363.9344
6   2013     5    15      9E        -3       41      213 311.7073
7   2013     5    15      9E        -7       43      187 260.9302
8   2013     5    15      9E       -13       99      659 399.3939
9   2013     5    15      9E        -6       35       94 161.1429
10  2013     5    15      9E       -12       84      549 392.1429
# ... with more rows

In [13]:
collect(c4)

year,month,day,carrier,dep_delay,air_time,distance,speed
2013,5,15,9E,-10,91,569,375.1648
2013,5,15,9E,-9,69,340,295.6522
2013,5,15,9E,-1,87,509,351.0345
2013,5,15,9E,-8,161,1029,383.4783
2013,5,15,9E,-3,122,740,363.9344
2013,5,15,9E,-3,41,213,311.7073
2013,5,15,9E,-7,43,187,260.9302
2013,5,15,9E,-13,99,659,399.3939
2013,5,15,9E,-6,35,94,161.1429
2013,5,15,9E,-12,84,549,392.1429


In [14]:
c4$query

NULL

In [15]:
explain(c4) 

<SQL>
SELECT *
FROM (SELECT `year`, `month`, `day`, `carrier`, `dep_delay`, `air_time`, `distance`, `distance` / `air_time` * 60.0 AS `speed`
FROM (SELECT `year` AS `year`, `month` AS `month`, `day` AS `day`, `carrier` AS `carrier`, `dep_delay` AS `dep_delay`, `air_time` AS `air_time`, `distance` AS `distance`
FROM (SELECT *
FROM `flights`
WHERE ((`year` = 2013.0) AND (`month` = 5.0) AND (`day` = 15.0)))))
ORDER BY `year`, `month`, `day`, `carrier`


<PLAN>
   addr       opcode p1    p2 p3           p4 p5 comment
1     0         Init  0    56  0              00      NA
2     1   SorterOpen  4     9  0       k(1,B) 00      NA
3     2     OpenRead  3   100  0           19 00      NA
4     3     OpenRead  5 22625  0     k(4,,,,) 02      NA
5     4         Real  0     1  0         2013 00      NA
6     5         Real  0     2  0            5 00      NA
7     6         Real  0     3  0           15 00      NA
8     7     Affinity  1     3  0          DDD 00      NA
9     8       SeekGE  5  

In [16]:
translate_sql(x)
translate_sql("x")
translate_sql(x == 1 && (y < 2 || z > 3))
translate_sql(x ^ 2 < 10)

<SQL> "x"

<SQL> 'x'

<SQL> "x" = 1.0 AND ("y" < 2.0 OR "z" > 3.0)

<SQL> POWER("x", 2.0) < 10.0

In [17]:
translate_sql(x %% 2 == 10)
translate_sql(1)
translate_sql(1L)

<SQL> "x" % 2.0 = 10.0

<SQL> 1.0

<SQL> 1

In [18]:
translate_sql(glob(x, y))
translate_sql(x %like% "ab*")

<SQL> GLOB("x", "y")

<SQL> "x" LIKE 'ab*'

In [19]:
by_tailnum <- group_by(flights_sqlite, tailnum)
delay <- summarise(by_tailnum,
  count = n(),
  dist = mean(distance),
  delay = mean(arr_delay)
)
delay <- filter(delay, count > 20, dist < 2000)
delay_local <- collect(delay)

In [20]:
by_tailnum

Source:   query [?? x 19]
Database: sqlite 3.11.1 [/tmp/RtmpJZUcZp/nycflights13.sqlite]
Groups: tailnum

    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1   2013     1     1      517            515         2      830            819
2   2013     1     1      533            529         4      850            830
3   2013     1     1      542            540         2      923            850
4   2013     1     1      544            545        -1     1004           1022
5   2013     1     1      554            600        -6      812            837
6   2013     1     1      554            558        -4      740            728
7   2013     1     1      555            600        -5      913            854
8   2013     1     1      557            600        -3      709            723
9   2013     1     1      557            600        -3      838            846
10  2013     1     1      

In [21]:
delay

Source:   query [?? x 4]
Database: sqlite 3.11.1 [/tmp/RtmpJZUcZp/nycflights13.sqlite]

   tailnum count     dist      delay
     <chr> <int>    <dbl>      <dbl>
1     <NA>  2512 710.2576         NA
2   N0EGMQ   371 676.1887  9.9829545
3   N10156   153 757.9477 12.7172414
4   N102UW    48 535.8750  2.9375000
5   N103US    46 535.1957 -6.9347826
6   N104UW    47 535.2553  1.8043478
7   N10575   289 519.7024 20.6914498
8   N105UW    45 524.8444 -0.2666667
9   N107US    41 528.7073 -5.7317073
10  N108UW    60 534.5000 -1.2500000
# ... with more rows

In [22]:
delay_local

tailnum,count,dist,delay
NA,2512,710.2576,NA
N0EGMQ,371,676.1887,9.9829545
N10156,153,757.9477,12.7172414
N102UW,48,535.8750,2.9375000
N103US,46,535.1957,-6.9347826
N104UW,47,535.2553,1.8043478
N10575,289,519.7024,20.6914498
N105UW,45,524.8444,-0.2666667
N107US,41,528.7073,-5.7317073
N108UW,60,534.5000,-1.2500000
